In [21]:
import pandas as pd
import numpy as np

In [22]:
has_parent = pd.read_csv('2022bkk-buget-has_parent.csv')
item = pd.read_csv('2022bkk-buget-item.csv', index_col='id')

In [23]:
df = item.join(has_parent.set_index('id'), on='id')

In [24]:
df

,text,amount,filename,page,line_num,parent
id,,,,,,
1,งานอํานวยการและบริหารสํานักงานเขต 10350780 บาท,10350780,65026.pdf,22,2,0
2,1. งบบุคลากร 5716350 บาท,5716350,65026.pdf,22,3,1
3,1.1 เงินเดือน 3648750 บาท,3648750,65026.pdf,22,4,2
4,01101-2 อัตราเดิม 10 อัตรา 3178850 บาท,3178850,65026.pdf,22,5,3
5,01102-1 เงินเลื่อนขั้น 190750 บาท,190750,65026.pdf,22,6,3
...,...,...,...,...,...,...
28027,07109-1 (4) ค่าใช้จ่ายในการพัฒนาคุณภาพการดําเน...,29800,65075.pdf,62,3,28023
28028,07124-1 (5) ค่าใช้จ่ายในการจัดประชุมสัมมนาคณะก...,81300,65075.pdf,62,5,28023
28029,07125-1 (6) ค่าใช้จ่ายในการสัมมนาประธานกรรมการ...,21300,65075.pdf,62,7,28023


In [25]:

psum = df.groupby('parent').amount.sum()
psumb = df.loc[df.parent.drop(df[df.parent == 0].index).unique()].amount.agg(lambda x: x == psum.loc[x.index])

id
1        True
2        True
3        True
10       True
15       True
         ... 
28012    True
28013    True
28017    True
28020    True
28023    True
Name: amount, Length: 8789, dtype: bool

In [26]:
df_ = df.assign(is_equal=df.apply(lambda x: psumb.loc[x.parent if x.parent != 0 else x.name], axis=1))

In [27]:
df_[df_.is_equal == False]#.to_csv('toFix.csv')

,text,amount,filename,page,line_num,parent,is_equal
id,,,,,,,
1297,01101-1 อัตราเดิม 4 อัตรา 1222120 บาท,1222120,65028.pdf,36,4,1296,False
1298,01102-1 เงินเลื่อนขั้น 12120 บาท,12120,65028.pdf,36,5,1296,False
1299,01107-1 เงินค่าตอบแทนรายเดือนของข้าราชการ 1680...,16800,65028.pdf,36,6,1296,False
1300,01108-1 เงินเพิ่มการครองชีพชั่วคราวของข้าราชกา...,2840,65028.pdf,36,7,1296,False
1301,01109-1 เงินช่วยเหลือค่าครองชีพของข้าราชการ 36...,360,65028.pdf,36,8,1296,False
...,...,...,...,...,...,...,...
27440,3. งบลงทุน 2620200 บาท,2620200,65074.pdf,65,2,27415,False
27455,4. งบเงินอุดหนุน 38824800 บาท,38824800,65074.pdf,67,27,27415,False
27458,5. งบรายจ่ายอื่น 5482620 บาท,5482620,65074.pdf,68,1,27415,False
